In [1]:
import graphlab
import numpy as np
import pandas as pd
# import matplotlib 

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
# Drop the duplicates 
Total_Test = pd.read_csv("../Total_Test_AfterProcessing.csv")
Total_Test = Total_Test.drop_duplicates('answer_id')
Total_Test = Total_Test.fillna(0)
Total_Test['question_id'] = Total_Test['question_id'].astype(str)
Total_Test['answer_voted'] = Total_Test['answer_voted'].astype(float)
# Total_Test # 42512 rows × 18 columns

In [3]:
Total_Test.to_csv('./Total_Test.csv', encoding='utf-8', index = False)
Total_Test= pd.read_csv('./Total_Test.csv')
# Total_Test  # 42512 rows × 18 columns

In [4]:
# Total_Test  # 42512 rows × 18 columns
been_voted = Total_Test[ Total_Test['answer_voted'] > 0 ]
un_voted = Total_Test[ Total_Test['answer_voted'] == 0 ]
been_voted = been_voted.drop_duplicates('question_id')
been_voted.to_csv('./Been_Voted.csv', encoding='utf-8', index = False)
# been_voted  # 17039 rows × 18 columns
# un_voted    # 14823 rows × 18 columns

In [5]:
been_voted = pd.read_csv('./Been_Voted.csv')
been_voted_qid = been_voted['question_id']
# len(been_voted_qid)   # 17039
# type(been_voted_qid)  # Series
# been_voted_qid[10]

In [6]:
# been_voted_qid

In [7]:
# Our Strategy is Select Qid in Voted Question List, 
# Select Similar Question From Total Test Set 

In [8]:
question_content = Total_Test[['question_id','question_content','question_tags']]
# question_content  # 42512 rows × 3 columns
question_content = question_content.drop_duplicates('question_id')
# question_content # 31862 rows × 3 columns
question_content.to_csv('./Question_Content.csv', encoding='utf-8', index = False)

In [9]:
question_content = graphlab.SFrame.read_csv('./Question_Content.csv')
# question_content  # 31862 rows x 3 columns

[INFO] 1461143637 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /home/zpgao/anaconda2/lib/python2.7/site-packages/certifi/cacert.pem
1461143637 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to beyond_acm@163.com and will expire on January 04, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-8480 - Server binary: /home/zpgao/anaconda2/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1461143637.log
[INFO] GraphLab Server Version: 1.7.1


PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/SIMILAR/Question_Content.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.133862 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /home/zpgao/ML/Best_Answer/SVM_RANK/SIMILAR/Question_Content.csv
PROGRESS: Parsing completed. Parsed 31862 lines in 0.05849 secs.


In [10]:
# add 'text' && 'word_count' && 'tfidf' features
question_content['text'] = question_content['question_tags'] + question_content['question_content']
question_content['word_count'] = graphlab.text_analytics.count_words(question_content['text'])

In [11]:
tfidf = graphlab.text_analytics.tf_idf(question_content['word_count'])

In [12]:
question_content['tfidf'] = tfidf

In [13]:
# been_voted_qid
type ( been_voted_qid[99] )

numpy.int64

In [14]:
# question_content[ question_content['question_id'] == been_voted_qid[5] ]

In [16]:
# question_content
# len(been_voted)  17039

In [17]:
def Select_Question( num_question ):
    # Initial selected_question
    selected_question = question_content[ question_content['question_id'] == been_voted_qid[7] ]
    for i in range(num_question) :
        # Random Select Index 
        selected_qid = been_voted_qid[i*134 + 77]
        selected_content = question_content[ question_content['question_id'] == selected_qid ]
        selected_question = selected_question.append(selected_content)
    return selected_question

In [18]:
# Random Select 50 voted / Unvoted Question
selected = Select_Question(99)

In [19]:
# len( selected )
# selected 

In [20]:
knn_model = graphlab.nearest_neighbors.create(question_content, features=['tfidf'], label='question_id')

PROGRESS: Starting brute force nearest neighbors model training.


In [21]:
Similar_Qid_List = {}
for i in range( len(selected) ) :
    key = 'Q' + str(i) + '_Similar_List'
    Q_i = selected[i:i+1]
    value = knn_model.query(Q_i, k=5)['reference_label']
    Similar_Qid_List[key] = value

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00313853  | 2.777ms      |
PROGRESS: | Done         |         | 100         | 24.675ms     |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00313853  | 2.468ms      |
PROGRESS: | Done         |         | 100         | 23.92ms      |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | #

In [25]:
# been_voted_qid[134 + 77]

In [26]:
# Similar_Qid_List

In [27]:
def Get_Index_QuestionID(qid) :
    return Total_Test[ Total_Test['question_id'] == qid].index.values

In [28]:
# print Get_Index_QuestionID( 1006907 )

In [29]:
def Get_IndexList(Q_List) :
    IndexList = []
    for qid in Q_List:
        indexes = Get_Index_QuestionID( qid )
        if len(indexes) == 0:
            indexes = Get_Index_QuestionID( qid )
        # print qid, indexes
        for index in indexes:
            IndexList.append(index)
            # print len( IndexList )
    return IndexList

In [30]:
# Similar_Qid_List

In [31]:
# From Similar Qid List Found Similar Index 
Similar_Index_List = {}
for i in range( len(Similar_Qid_List) ) :
    key = 'Q' + str(i) + '_Similar_List'
    # value is a list of similar qid
    Qid_List = Similar_Qid_List[key]
    Index_List = Get_IndexList(Qid_List)
    Similar_Index_List[key]  = Index_List    

In [32]:
# Similar_Index_List
# len( Similar_Index_List )  # 50

In [33]:
# import json
# import pickle

def writeDict(dict, filename, sep):
    with open(filename, "w") as f:
#        for key in dict.keys():            
#            f.write(str(key) + ":" + str(dict[key]) + "\n")
        for key, value in sorted( dict.items() ):
            f.write( str(key) + sep + ','.join(str(x) for x in value) + '\n' )

In [34]:
def readDict(filename, sep):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.split(sep)
            key = values[0]
            value = values[1].split(',')
            #dict[values[0]] = {int(x) for x in values[1:len(values)]}
            dict[key] = value
        return(dict)

In [35]:
writeDict(Similar_Qid_List, './Similar_Qid_List.txt', ':')
writeDict(Similar_Index_List, './Similar_Index_List.txt', ':')

In [36]:
test = readDict('./Similar_Qid_List.txt', ':')

In [37]:
test == Similar_Qid_List

True